In [ ]:
import cv2
from super_gradients.common.object_names import Models
from super_gradients.training import models
import numpy as np
import urllib

def open_input(input_path):
    """
    Abre a entrada com base no tipo (imagem, vídeo ou streaming) e retorna
    o objeto VideoCapture correspondente.
    """
    if input_path.startswith('http'):  # Se for um link de streaming
        stream = urllib.request.urlopen(input_path)
        bytes = bytearray()
        while True:
            bytes += stream.read(1024)
            a = bytes.find(b'\xff\xd8')
            b = bytes.find(b'\xff\xd9')
            if a != -1 and b != -1:
                jpg = bytes[a:b + 2]
                bytes = bytes[b + 2:]
                frame = cv2.imdecode(np.fromstring(jpg, dtype=np.uint8), cv2.IMREAD_COLOR)
                return cv2.VideoCapture(frame)
    elif input_path.endswith(('jpg', 'jpeg', 'png', 'bmp')):  # Se for uma imagem
        return cv2.VideoCapture(input_path)
    else:  # Se for um vídeo
        return cv2.VideoCapture(input_path)

def process_input(input_path):
    """
    Processa a entrada com base no tipo (imagem, vídeo ou streaming) e retorna
    o objeto VideoCapture correspondente e a largura e altura dos quadros.
    """
    cap = open_input(input_path)

    # Obtém a largura e a altura do quadro do vídeo
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    return cap, frame_width, frame_height

# Define o caminho da entrada (imagem, vídeo ou link de streaming)
INPUT_PATH = "/kaggle/input/video/videoplayback (1).mp4"

# Processa a entrada com base no tipo
cap, frame_width, frame_height = process_input(INPUT_PATH)

# Define o codec e cria o objeto VideoWriter
output_path = "/kaggle/working/output_video.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, 20.0, (frame_width, frame_height))

# Carrega o modelo YOLO-NAS-L
model = models.get(Models.YOLO_NAS_L, pretrained_weights="coco")

# Define o limite de confiança inicial
confidence_threshold = 0.2

# Cria um arquivo de log para salvar as previsões
log_file = open("/kaggle/working/predictions_log.txt", "w")

# Processa cada frame do vídeo
frame_count = 0
while cap.isOpened() and frame_count < 1000:
    ret, frame = cap.read()
    if not ret:
        break

    # Processa o quadro e obtém as previsões
    frame_processed, frame_predictions = process_frame(frame)

    # Escreve o quadro processado no vídeo de saída
    out.write(frame_processed)

    # Salva as previsões no arquivo de log
    for prediction in frame_predictions:
        log_file.write(f"Class: {prediction['class_name']}, Confidence: {prediction['confidence']}, BBox: {prediction['bbox']}\n")

    frame_count += 1

# Fecha o arquivo de log
log_file.close()

# Libera os objetos de captura de vídeo e gravação
cap.release()
out.release()

print("Vídeo com previsões salvo com sucesso em:", output_path)
print("Arquivo de log com as previsões salvo com sucesso.")


[2024-05-11 13:18:13] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2024-05-11 13:18:13] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l
[2024-05-11 13:18:13] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 13:18:15] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 13:18:16] INFO - pipelines.py - Fusing some of the model's layers. If this takes too much memory, you can deactivate it by setting `fuse_model=False`
[2024-05-11 13:18:17] INFO - pipelines.py - Fusing some of the mod